# Elasticsearch for .NET developers - introduction

Elasticsearch is a powerful and flexible search and analytics engine that can greatly benefit .NET developers looking to enhance their applications.

While this summary is not a substitute for official documentation, it serves as a helpful introduction to Elasticsearch's key features and how they can benefit .NET developers.

To demonstrate Elasticsearch's capabilities interactively, a [polyglot notebook][1] was generated. This notebook functions as an interactive document that integrates code execution, textual content, visualizations, and multimedia elements into a unified and cohesive environment.

Currently, this document won't go into details on various infrastructure related tasks such as configuring Elasticsearch nodes, managing shards and other similar operational activities, but these subjects may be part of a future update.

[1]: <https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode> "Polyglot Notebook"

## Getting Started

**NOTE**: This notebook assumes that you already have *docker* installed and if you are working on a windows machine you may also need to run some commands through *wsl*.

In order to run this notebook locally, you need to go through the following steps:

1. Install the latest [Visual Studio Code](https://code.visualstudio.com/).
2. Install the latest [.NET 7 SDK](https://dotnet.microsoft.com/en-us/download).
3. Install the Polyglot Notebooks extension from the [marketplace](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).
4. Create a `docker-compose.yml` file and add the following content:

```yml
version: '3.7'

services:
    elasticsearch:
        image: docker.elastic.co/elasticsearch/elasticsearch:8.5.0
        container_name: elasticsearch
        environment: 
            - xpack.security.enabled=false
            - discovery.type=single-node
        ulimits:
            memlock:
                soft: -1
                hard: -1
            nofile:
                soft: 65536
                hard: 65536
        ports:
            - 9200:9200
            - 9300:9300
            
    kibana:
        container_name: kibana
        image: docker.elastic.co/kibana/kibana:8.5.0
        environment: 
            - ELASTICSEARCH_HOSTS=http://elasticsearch:9200
        ports:
            - 5601:5601
        depends_on:
            - elasticsearch
```

5. Run the following command: `docker compose up -d` in the directory where the `docker-compose.yml` file was created.

If you encounter the following error while starting the Elasticsearch container: `max file descriptors [65000] for elasticsearch process is too low, increase to at least [65536]` run the following command: `sudo sysctl -w vm.max_map_count=262144`.

## What is Elasticsearch?

Elasticsearch is a distributed document store. Instead of storing information as rows of columnar data, Elasticsearch stores complex data structures that have been serialized as JSON **documents** in an **index**.

An **index** can be thought of as an optimized collection of **documents** and each **document** is a collection of **fields**, which are the key-value pairs that contain your data.

Elasticsearch also has the ability to be schema-less, which means that documents can be indexed without explicitly specifying how to handle each of the different fields that might occur in a document. When dynamic **mapping** is enabled, Elasticsearch automatically detects and adds new **fields** to the index.

Ultimately, however, you know more about your data and how you want to use it than Elasticsearch can. You can define rules to control dynamic **mapping** and explicitly define **mappings** to take full control of how **fields** are stored and indexed [1].

[1]. https://www.elastic.co/guide/en/elasticsearch/reference/current/documents-indices.html#documents-indices

## Working with the .NET client

In order to use the .NET Elasticsearch client you firstly need to add the NEST nuget package to the notebook:

In [1]:
#r "nuget: NEST, *-*"

Installed Packages NEST, 7.17.5

**NOTE**: NEST is still the recommended client for .NET until the new *Elastic.Clients.Elasticsearch v8* will have feature parity with the *NEST v7* client [2].

[2]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/current/installation.html

Before creating some settings and instantiating the client you also need to register some usings:

In [2]:
using System;
using Nest;

An instance of the `ElasticClient` is required before mapping, indexing, searching, etc.

In [3]:

var settings = new ConnectionSettings()     // the parameterless constructor defaults to http://localhost:9200
                .DisableDirectStreaming()   // this setting allows us to see the actual debug information and not some random bytes that were sent across the wire
                .PrettyJson();              // instead of seeing a minified version of the JSON data that was sent across the wire, we want to see a prettified version

var client = new ElasticClient(settings);

The POCO that will be used throughout this notebook can be declared by running the following code block:

In [4]:
public class Employee
{
    public string Id { get; set; }
    public string Name { get; set; }
    public int Salary { get; set; }
    public bool IsManager { get; set; }
    public DateTime Birthday { get; set; }
    public TimeSpan Hours { get; set; }
}

### Mappings

#### Auto mapping

When creating a mapping either when creating an index or through the Put Mapping API, NEST offers a feature called auto mapping that can automagically infer the correct Elasticsearch field datatypes from the CLR POCO property types you are mapping.

In order to offer additional information and examine the request payload that was sent to Elasticsearch, the debug information of the request will also be provided [3]. 

[3]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/7.17/auto-map.html

In [5]:
var createIndexResponse = client.Indices.Create(
    "employees-index-1", 
    c => c.Map(m => m.AutoMap<Employee>())
);

if (!createIndexResponse.IsValid)
{
    // If the request isn't valid, we can take action here
}

createIndexResponse.DebugInformation

Valid NEST response built from a successful (200) low level call on PUT: /employees-index-1?pretty=true
# Audit trail of this API call:
 - [1] ProductCheckOnStartup: Took: 00:00:00.1332027
 - [2] ProductCheckSuccess: Node: http://localhost:9200/ Took: 00:00:00.1224710
 - [3] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.4901082
# Request:
{"mappings":{"properties":{"id":{"fields":{"keyword":{"ignore_above":256,"type":"keyword"}},"type":"text"},"name":{"fields":{"keyword":{"ignore_above":256,"type":"keyword"}},"type":"text"},"salary":{"type":"integer"},"isManager":{"type":"boolean"},"birthday":{"type":"date"},"hours":{"type":"long"}}}}
# Response:
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "employees-index-1"
}



In [6]:
// if you want to view the whole response object, uncomment the following line:
// createIndexResponse

// if you want to run the create command multiple times, you may have to delete the previously created index first 
//client.Indices.Delete("employees-index-1");

If you take a look at the response shown after running the previous code block, you can see that the debug information contains a `Request` section and a `Response` section. The `Request` section contains the mappings that were generated by the Elasticsearch .NET client.

Although this can help you get started working with Elasticsearch, most use cases require predefined mappings.

#### Attribute mapping

In Auto mapping, you saw that the type mapping for a POCO can be inferred from the properties of the POCO, using `.AutoMap()`. But what do you do when you want to map differently to the inferred mapping? This is where attribute mapping can help [4].

[4]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/7.17/attribute-mapping.html#attribute-mapping

In [7]:
public class Employee
{
    [Keyword]
    public string Id { get; set; }

    [Text(Name = "full_name")]
    public string Name { get; set; }

    [Number(Coerce = true)]
    public int Salary { get; set; }

    [Boolean(NullValue = false)]
    public bool IsManager { get; set; }

    [Date]
    public DateTime Birthday { get; set; }

    [Text(Name = "work_week_hours")]
    public TimeSpan Hours { get; set; }
}

The field types will be presented later within this document, however, it's important to note that, at present, the `text` field type enables partial matching during Elasticsearch searches. On the other hand, the `keyword` field type does not support partial matching. Given this, it is recommended to consider altering the field type of the `Id` property to `keyword` while indexing the `Name` property solely as `text`, especially if one of the requested use cases is to search for employees based on either the first name or the last name.

Indexing a field as both `keyword` and `text` in Elasticsearch can have performance and storage implications, and it may not provide significant benefits in most cases.

In [8]:
var createIndexResponse = client.Indices.Create(
    "employees-index-2", 
    c => c.Map(m => m.AutoMap<Employee>())
);

if (!createIndexResponse.IsValid)
{
    // If the request isn't valid, we can take action here
}

createIndexResponse.DebugInformation

Valid NEST response built from a successful (200) low level call on PUT: /employees-index-2?pretty=true
# Audit trail of this API call:
 - [1] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.3105369
# Request:
{"mappings":{"properties":{"id":{"type":"keyword"},"full_name":{"type":"text"},"salary":{"coerce":true,"type":"float"},"isManager":{"null_value":false,"type":"boolean"},"birthday":{"type":"date"},"work_week_hours":{"type":"text"}}}}
# Response:
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "employees-index-2"
}



In [9]:
// if you want to view the whole response object, uncomment the following line:
// createIndexResponse

// if you want to run the create command multiple times, you may have to delete the previously created index first 
//client.Indices.Delete("employees-index-2");

As you can see in the `Request` section above, the `Employee` mapping is fairly different from the one generated by the Elasticsearch .NET client (and contains less data than the previous `Request`), mainly because now you can specify whether the type of the field a string maps to is `keyword` or `text`.

Attribute mapping can be a convenient way to control how POCOs are mapped with minimal code, however there are some mapping features that cannot be expressed with attributes.

#### Fluent mapping

Fluent mapping POCO properties to fields within an Elasticsearch type mapping offers the most control over the process. With fluent mapping, each property of the POCO is explicitly mapped to an Elasticsearch type field mapping [5].

[5]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/7.17/fluent-mapping.html

In order to override the previous class declaration that contained attributes required for the attribute mapping you need to run the following code block:

In [10]:
public class Employee
{
    public string Id { get; set; }
    public string Name { get; set; }
    public int Salary { get; set; }
    public bool IsManager { get; set; }
    public DateTime Birthday { get; set; }
    public TimeSpan Hours { get; set; }
}

To create a mapping for the `Employee` type, the fluent API can be used to map each property explicitly:

In [11]:
var createIndexResponse = client.Indices.Create(
    "employees-index-3", c => c
    .Map<Employee>(m => m
        .Properties(ps => ps
            .Keyword(f => f
                .Name(e => e.Id))
            .Text(f => f
                .Name(e => e.Name))
            .Number(f => f
                .Name(e => e.Salary)
                .Type(NumberType.Integer))                
            .Boolean(f => f
                .Name(e => e.IsManager))
            .Date(f => f
                .Name(e => e.Birthday))
            .Text(f => f
                .Name(e => e.Hours))
        )
    )
);

if (!createIndexResponse.IsValid)
{
    // If the request isn't valid, we can take action here
}

createIndexResponse.DebugInformation

Valid NEST response built from a successful (200) low level call on PUT: /employees-index-3?pretty=true
# Audit trail of this API call:
 - [1] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.3526981
# Request:
{"mappings":{"properties":{"id":{"type":"keyword"},"name":{"type":"text"},"salary":{"type":"integer"},"isManager":{"type":"boolean"},"birthday":{"type":"date"},"hours":{"type":"text"}}}}
# Response:
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "employees-index-3"
}



In [12]:
// if you want to view the whole response object, uncomment the following line:
// createIndexResponse

// if you want to run the create command multiple times, you may have to delete the previously created index first 
//client.Indices.Delete("employees-index-3");

### Indexing documents

NEST exposes the index and bulk APIs of Elasticsearch as methods, to enable indexing of single or multiple documents. In addition to this, the client provides some convenient shorthand methods for the typical indexing approaches [6].

[6]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/7.17/indexing-documents.html

In [13]:
var employee = new Employee()
{
    Id = "id-1",
    Name = "John Carmack",
    Birthday = new DateTime(1970, 08, 21),
    IsManager = true,
    Hours = TimeSpan.FromHours(40),
    Salary = 200000
};

In [14]:
// fluent syntax
var indexResponse = client.Index(employee, i => i.Index("employees-index-3")); 
if (!indexResponse.IsValid)
{
    // If the request isn't valid, we can take action here
}

indexResponse.DebugInformation

Valid NEST response built from a successful (201) low level call on PUT: /employees-index-3/_doc/id-1?pretty=true
# Audit trail of this API call:
 - [1] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.0531524
# Request:
{"id":"id-1","name":"John Carmack","salary":200000,"isManager":true,"birthday":"1970-08-21T00:00:00","hours":1440000000000}
# Response:
{
  "_index" : "employees-index-3",
  "_id" : "id-1",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 0,
  "_primary_term" : 1
}



In [15]:
var employee = new Employee()
{
    Name = "Tom Hall",
    Birthday = new DateTime(1964, 09, 02),
    IsManager = true,
    Hours = TimeSpan.FromHours(40),
    Salary = 100000
};

In [16]:
// object initializer syntax and setting the id as part of the request
var indexResponse = client.Index(new IndexRequest<Employee>(employee, "employees-index-3", "id-2"));
indexResponse.DebugInformation

Valid NEST response built from a successful (201) low level call on PUT: /employees-index-3/_doc/id-2?pretty=true
# Audit trail of this API call:
 - [1] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.0155244
# Request:
{"name":"Tom Hall","salary":100000,"isManager":true,"birthday":"1964-09-02T00:00:00","hours":1440000000000}
# Response:
{
  "_index" : "employees-index-3",
  "_id" : "id-2",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 1,
  "_primary_term" : 1
}



In [17]:
// if you want to view the whole response object, uncomment the following line:
// indexResponse

**NOTE**: Executing the index command repeatedly without modifying the id of the document that is being indexed will result in the provided document being updated within Elasticsearch.

Multiple documents can be indexed using the `IndexMany` and `IndexManyAsync` methods, but for very large document collection it is recommended to use the `BulkAllObservable` helper instead [7].

[7]. https://www.elastic.co/guide/en/elasticsearch/client/net-api/7.17/indexing-documents.html#_multiple_documents_with_indexmany

In [18]:
var employees = new Employee[]
{
    new Employee
    {
        Id = "id-3",
        Name = "Adrian Carmack",
        Birthday = new DateTime(1970, 08, 24),
        IsManager = false,
        Hours = TimeSpan.FromHours(40),
        Salary = 200000
    }, 
    new Employee
    {
        Id = "id-4",
        Name = "John Romero",
        Birthday = new DateTime(1969, 05, 05),
        IsManager = false,
        Hours = TimeSpan.FromHours(40),
        Salary = 100000
    },
};

In [19]:
var indexManyResponse = client.IndexMany(employees, "employees-index-3"); 

if (indexManyResponse.Errors) 
{
    foreach (var itemWithError in indexManyResponse.ItemsWithErrors) 
    {
        Console.WriteLine($"Failed to index document {itemWithError.Id}: {itemWithError.Error}");
    }
}

indexManyResponse.DebugInformation

Invalid NEST response built from a successful (200) low level call on POST: /employees-index-3/_bulk?pretty=true
# Invalid Bulk items:
  operation[0]: index returned 201 _index: employees-index-3 _type:  _id: id-3 _version: 1 error: 
  operation[1]: index returned 201 _index: employees-index-3 _type:  _id: id-4 _version: 1 error: 
# Audit trail of this API call:
 - [1] HealthyResponse: Node: http://localhost:9200/ Took: 00:00:00.0565744
# Request:
{"index":{"_id":"id-3"}}
{"id":"id-3","name":"Adrian Carmack","salary":200000,"isManager":false,"birthday":"1970-08-24T00:00:00","hours":1440000000000}
{"index":{"_id":"id-4"}}
{"id":"id-4","name":"John Romero","salary":100000,"isManager":false,"birthday":"1969-05-05T00:00:00","hours":1440000000000}

# Response:
{
  "took" : 12,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "employees-index-3",
        "_id" : "id-3",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" :

In [20]:
// if you want to view the whole response object, uncomment the following line:
// indexManyResponse

**NOTE**: Although the message from the previous code block may be `Invalid NEST response built from a successful (200) low level call on POST` the request went through and the provided data was indexed successfully.

If you require more control over indexing many documents, you can use the `Bulk` and `BulkAsync` methods and use the descriptors to customize the bulk calls.

### Searching

By default, Elasticsearch sorts matching search results by relevance score, which measures how well each document matches a query.

#### Query context

In the query context, a query clause answers the question *“How well does this document match this query clause?”* Besides deciding whether or not the document matches, the query clause also calculates a relevance score in the `_score` metadata field.

Query context is in effect whenever a query clause is passed to a query parameter, such as the `query` parameter in the search API.

The simplest of queries is the `match_all` query; this will return all documents, giving them all a `_score` of 1.0 [8].

[8]. https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#query-context

In [28]:
var searchResponse = client.Search<Employee>(s => s
    .Index("employees-index-3")
    .Query(q => q
        .MatchAll()
    )
);

var documents = searchResponse.Hits.Select(x => new { x.Id, x.Score, x.Source, x.Source.Hours.TotalHours });

documents

index Id Score Source TotalHours 0 id-1 1 Id Name Salary IsManager Birthday Hours id-1 John Carmack 200000 True 1970-08-21 00:00:00Z 1.16:00:00 40 1 id-2 1 Id Name Salary IsManager Birthday Hours <null> Tom Hall 100000 True 1964-09-02 00:00:00Z 1.16:00:00 40 2 id-3 1 Id Name Salary IsManager Birthday Hours id-3 Adrian Carmack 200000 False 1970-08-24 00:00:00Z 1.16:00:00 40 3 id-4 1 Id Name Salary IsManager Birthday Hours id-4 John Romero 100000 False 1969-05-05 00:00:00Z 1.16:00:00 40

**NOTE**: As you can see in the cell output above, the id of a certain object doesn't have to be be populated in order to index a document with a specific id. However, it's important to note that this practice is not particularly recommended.

In [22]:
// if you want to view the whole response object, uncomment the following line:
// searchResponse

The following query should return a list of documents that match or partially match the provided name:

In [23]:
var searchResponse = client.Search<Employee>(s => s
    .Index("employees-index-3")
    .Query(q => q
        .Match(m => m
            .Field(x => x.Name)
            .Query("John Carmack"))
    )
);

var documents = searchResponse.Hits.Select(x => new { x.Id, x.Score, x.Source, x.Source.Hours.TotalHours });

documents

index Id Score Source TotalHours 0 id-1 1.3862942 Id Name Salary IsManager Birthday Hours id-1 John Carmack 200000 True 1970-08-21 00:00:00Z 1.16:00:00 40 1 id-3 0.6931471 Id Name Salary IsManager Birthday Hours id-3 Adrian Carmack 200000 False 1970-08-24 00:00:00Z 1.16:00:00 40 2 id-4 0.6931471 Id Name Salary IsManager Birthday Hours id-4 John Romero 100000 False 1969-05-05 00:00:00Z 1.16:00:00 40

Since the previous query was executed in the query context, the results are sorted by their relevance score:
- "John Carmack" fully matches the query and has the highest score
- "Adrian Carmack" partially matches the query and has a lower score
- "John Romero" partially matches the query and has a lower score
- "Tom Hall" doesn't match the query at all and it's not part of the result set 

In [24]:
// if you want to view the whole response object, uncomment the following line:
// searchResponse


#### Filter context

In a filter context, a query clause answers the question “Does this document match this query clause?” The answer is a simple Yes or No — no scores are calculated. Filter context is mostly used for filtering structured data, e.g.

- Does this timestamp fall into the range 2015 to 2016?
- Is the status field set to "published"?

Filter context is in effect whenever a query clause is passed to a `filter` parameter, such as the `filter` or `must_not` parameters in the `bool` query, the `filter` parameter in the `constant_score` query, or the `filter` aggregation [9].

[9]. https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#filter-context

The following query should return a list of documents for which the value of the `salary` property is less than *150000*:

In [25]:
var searchResponse = client.Search<Employee>(s => s
    .Index("employees-index-3")
    .Query(q => q
        .Range(r => r
            .Field(x => x.Salary)
            .LessThan(150000)
        )
    )
);

var documents = searchResponse.Hits.Select(x => new { x.Id, x.Score, x.Source, x.Source.Hours.TotalHours });

documents

index Id Score Source TotalHours 0 id-2 1 Id Name Salary IsManager Birthday Hours <null> Tom Hall 100000 True 1964-09-02 00:00:00Z 1.16:00:00 40 1 id-4 1 Id Name Salary IsManager Birthday Hours id-4 John Romero 100000 False 1969-05-05 00:00:00Z 1.16:00:00 40

As seen in the result from above, the score of the returned documents is constant and is not influenced by the value of the `salary` property. The result set simply answers the question: *Which employees have a salary lower than 150000?*

In [26]:
// if you want to view the whole response object, uncomment the following line:
// searchResponse

In [27]:
// if you want to rerun all the codeblocks without any errors you may need to delete all elasticsearch indexes:
// client.Indices.Delete("employees-index-1");
// client.Indices.Delete("employees-index-2");
// client.Indices.Delete("employees-index-3");

This document merely begins to explore a handful of the extensive capabilities that Elasticsearch has to offer. For an in depth understanding of any of the aforementioned features you should consult the official documentation.

Since this notebook was intended as a short introduction to Elasticsearch for .NET developers, additional 'per-feature' notebooks will be created in the future.